You may find this series of notebooks at https://github.com/databricks-industry-solutions/anti-money-laundering. For more information about this solution accelerator, visit https://www.databricks.com/blog/2021/07/16/aml-solutions-at-scale-using-databricks-lakehouse-platform.html.


# Data Deduplication

The last category of problems in AML that we focus on is entity resolution. Many open-source libraries tackle this problem, so for some basic entity fuzzy matching, we chose to highlight a library that achieves the linkage at scale. 

Developed by the ministry of Justice, [Splink](https://github.com/moj-analytical-services/splink) is an entity linkage framework that offers configurations to specify matching columns as well as blocking rules at enterprise scale. By combining field attributes (such as organization + mailing address), we could detect similarities and de-duplicate matching records. Splink works by assigning a match probability which can be used to identify transactions in which entity attributes are highly similar, indicating a potential alert in reported address, entity name, or transaction amounts. Given the fact that entity resolution can be highly manual for matching account information, having open-source libraries which automated this task and which save this information in Delta Lake can make investigators much more productive for case resolution. While there are several options available for entity matching like the one using Locality-Sensitive Hashing (LSH) as mentioned in this blog [post](https://databricks.com/blog/2021/05/24/machine-learning-based-item-matching-for-retailers-and-brands.html), we recommend our readers to identify the right algorithm for the job.

In [0]:
%pip install splink==2.1.14

  Obtaining dependency information for splink==2.1.14 from https://files.pythonhosted.org/packages/e4/b1/a48d8214fd3fbbb7a62e8efcce50489e74e3769f7a48ea827875a6912ef2/splink-2.1.14-py3-none-any.whl.metadata
  Obtaining dependency information for jsonschema<4.0,>=3.2 from https://files.pythonhosted.org/packages/c5/8f/51e89ce52a085483359217bc72cdbf6e75ee595d5b1d4b5ade40c7e018b8/jsonschema-3.2.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for sqlglot==1.23.1 from https://files.pythonhosted.org/packages/3d/94/16b706fc1dbded4213b9335e281d5d5ac118a11718d6afef58a1a6ef2f67/sqlglot-1.23.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/1.3 MB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━ 1.0/1.3 MB 36.6 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 1.3/1.3 MB 33.3 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/56.3 kB ? eta -:--:

In [0]:
dbutils.library.restartPython()

In [0]:
import re
import warnings
warnings.filterwarnings("ignore")
database_name = 'trishaldoshi_aml'

config = {
  'db_transactions': f"{database_name}.transactions",
  'db_entities': f"{database_name}.entities",
  'db_dedupe': f"{database_name}.dedupe",
  'db_synth_scores': f"{database_name}.synth_scores",
  'db_structuring': f"{database_name}.structuring",
  'db_structuring_levels': f"{database_name}.structuring_levels",
  'db_roundtrips': f"{database_name}.roundtrips",
  'db_risk_propagation': f"{database_name}.risk_propagation",
  'db_streetview': f"{database_name}.streetview",
  'db_dedupe_records': f"{database_name}.dedupe_splink",
}

In [0]:
raw_records = spark.read.table(config['db_dedupe'])
display(raw_records)

unique_id org_name address country amount postalcode 223254 The Bank of New York Mellon Corp. Canada Square, Canary Wharf, London, United Kingdom United Kingdom 2500 E145AL 223255 BNY Mellon Corp. Canary Wharf, London, UK UK 4500 E145AL 223256 The Bank of New York Mellon Corp. Canada Square, Canary Wharf, London, UK United Kingdom 2500 E145AL 223257 BNY Mellon Corp. Canary Wharf, London, UK UK 7800 E145AL

As reported above, we can easily eye ball some similarities between "The Bank of New York Mellon Corp." and "BNY Mellon". Obvious for a human eye, this becomes a real challenge at scale when doing so programmatically. Splink works by assigning a match probability which can be used to identify transactions in which entity attributes are highly similar, indicating a potential alert in reported address, entity name, or transaction amounts. To reduce the required number of pairwise comparisons among descriptions, methods for entity resolution typically perform a preprocessing step, called blocking, which places similar entity descriptions into blocks and executes comparisons only between descriptions within the same block.

In [0]:
settings = {
    "link_type": "dedupe_only",
    "blocking_rules": [
        "l.amount = r.amount",
    ],
    "comparison_columns": [
        {
            "col_name": "org_name",
            "term_frequency_adjustments": True},
        {
            "col_name": "address",
            "term_frequency_adjustments": True
        },
        {
            "col_name": "country"
        },
              {
            "col_name": "amount"
        }
    ]
}

from splink import Splink
linker = Splink(settings, raw_records, spark)
raw_records_entities = linker.get_scored_comparisons()
display(raw_records_entities.take(1))

match_weight match_probability unique_id_l unique_id_r org_name_l org_name_r gamma_org_name address_l address_r gamma_address country_l country_r gamma_country amount_l amount_r gamma_amount 14.79552948666091 0.9999648369489322 223254 223256 The Bank of New York Mellon Corp. The Bank of New York Mellon Corp. 1 Canada Square, Canary Wharf, London, United Kingdom Canada Square, Canary Wharf, London, UK 0 United Kingdom United Kingdom 1 2500 2500 1

As reported above, we quickly found some inconsistencies for NY Mellon address, with "Canada Square, Canary Wharf, London, United Kingdom" similar to "Canada Square, Canary Wharf, London, UK". We can store our de-duplicated records back to a delta table that can be used for AML investigation.

In [0]:
raw_records_entities.write.mode("overwrite").format("delta").saveAsTable(config['db_dedupe_records'])

In [0]:
temp_directory = "/Workspace/Users/trishaldoshi@outlook.com/aml_temp"

In [0]:
model = linker.model
model.probability_distribution_chart()
model.bayes_factor_chart()
model.all_charts_write_html_file(f"{temp_directory}/splink_charts.html", overwrite=True)

Let's dig deeper into some of statistics and plots offered by the Splink library. The following commands offer insights into how the match probability was calculated. Splink primarily uses a Expectation Maximization framework to maximize a likelihood function to generate match probabilities on record pairs as shown in the below graphs. Expectation Maximization is an iterative algorithm, so we can also see matches and non matches for different iterations as well. 

In [0]:
html_file_content = open(f"{temp_directory}/splink_charts.html", 'r').read()
displayHTML(html_file_content)

<!DOCTYPE html>

As we continue our exploratory data analysis we can also look at individual matches and respective explanations. In the below example we are matching on Org name and Address and the respective match probabilities are shown for these levels. Another good example would be to show matches on a person's name, address and deposit amounts to detect high volume of suspicious transactions.

In [0]:
from splink.intuition import intuition_report
row_dict = raw_records_entities.toPandas().sample(1).to_dict(orient="records")[0]
print(intuition_report(row_dict, model))


Initial probability of match (prior) = λ = 0.9999
------
Comparison of org_name.  Values are:               
org_name_l:                                        The Bank of New York Mellon Corp.
org_name_r:                                        The Bank of New York Mellon Corp.
Comparison has:                                    2 levels
Level for this comparison:                         gamma_org_name = 1
m probability = P(level|match):                    1
u probability = P(level|non-match):                1
Bayes factor = m/u:                                1
New probability of match (updated belief):         0.9999

------
Comparison of address.  Values are:                
address_l:                                         Canada Square, Canary Wharf, London, United Kingdom
address_r:                                         Canada Square, Canary Wharf, London, UK
Comparison has:                                    2 levels
Level for this comparison:                         gamma_ad

Finally, we can enrich our original dataset with a unique identifier that can be used to de-duplicate matching records. Once again, we do not wish to modify underlying data with AI, but rather provide our AML investigation team all the necessary context / utilities for them to take the necessary measures. 

In [0]:
df2 = spark.table(config['db_transactions'])
df2 = df2.withColumnRenamed("txn_id", "unique_id")
display(df2)

txn_dt unique_id originator_id beneficiary_id txn_amount originator_type beneficiary_type rptd_originator_address rptd_beneficiary_address rptd_originator_name rptd_beneficiary_name originator_bank_country beneficiary_bank_country 2021-05-18 100004 10000003 10000005 70.60757435220596 Company Company 2399 Delta Road, Tahoe, MD 29293 63828 Spark Lane, Reston, VA 22011 Inertia Bank DeltaTech US US 2021-05-22 100009 10000007 10000008 95.51511865763597 Company Company 2399 Delta Road, Tahoe, MD 29293 63828 Spark Lane, Reston, VA 22011 Inertia Bank DeltaTech US US 2021-05-21 100008 10000004 10000009 60.803507464350304 Company Company 2399 Delta Road, Tahoe, MD 29293 63828 Spark Lane, Reston, VA 22011 Inertia Bank DeltaTech US US 2021-05-17 100003 10000003 10000004 30.388144583541475 Company Company 2399 Delta Road, Tahoe, MD 29293 63828 Spark Lane, Reston, VA 22011 Inertia Bank DeltaTech US US 2021-05-16 100002 10000002 10000004 73.45955942780691 Company Company 2399 Delta Road, Tahoe, MD 29293 63828 Spark Lane, Reston, VA 22011 Inertia Bank DeltaTech US US 2021-05-20 100007 10000006 10000007 16.891239996921037 Company Company 2399 Delta Road, Tahoe, MD 29293 63828 Spark Lane, Reston, VA 22011 Inertia Bank DeltaTech US US 2021-05-15 100001 10000002 10000003 45.366186513288135 Company Company 2399 Delta Road, Tahoe, MD 29293 63828 Spark Lane, Reston, VA 22011 Inertia Bank DeltaTech US US 2021-05-19 100006 10000005 10000007 58.920669463295404 Company Company 2399 Delta Road, Tahoe, MD 29293 63828 Spark Lane, Reston, VA 22011 Inertia Bank DeltaTech US US 2021-05-15 100011 10000008 10000010 19.39768302907553 Company Company 2399 Delta Road, Tahoe, MD 29293 63828 Spark Lane, Reston, VA 22011 Inertia Bank DeltaTech US US 2021-05-19 223254 1223254 2223254 2500 Company Company Canada Square, Canary Wharf, London, United Kingdom Windsor Park, London, UK The Bank of New York Mellon Corp. Bank of America United Kingdom United Kingdom 2021-05-18 223255 1223255 2223255 4500 Company Company Canary Wharf, London, UK Windsor Park, London, UK BNY Mellon Corp. Bank of America UK UK 2021-05-19 223256 1223256 2223256 2500 Company Company Canada Square, Canary Wharf, London, UK Windsor Park, London, UK The Bank of New York Mellon Corp. Bank of America United Kingdom United Kingdom 2021-05-20 223257 1223257 2223257 7800 Company Company Canary Wharf, London, UK Windsor Park, London, UK BNY Mellon Corp. Bank of America UK UK 2021-05-14 100010 10000007 10000009 84.07930955396817 Company Company 2399 Delta Road, Tahoe, MD 29293 63828 Spark Lane, Reston, VA 22011 Inertia Bank DeltaTech US US 2021-05-18 100005 10000003 10000006 18.725937991950325 Company Company 2399 Delta Road, Tahoe, MD 29293 63828 Spark Lane, Reston, VA 22011 Inertia Bank DeltaTech US US 2021-05-14 100000 10000001 10000003 12.611258643876067 Company Company 2399 Delta Road, Tahoe, MD 29293 63828 Spark Lane, Reston, VA 22011 Inertia Bank DeltaTech US US 2021-05-15 1 1 2 5000 Person Company 120 Mason Lake Apt. 211
North Carmenport, OK 73475 93324 Hood Islands Apt. 113
Romerofort, UT 96153 Brenda Thomas Teresa Gibson US US 2021-05-16 2 3 4 5000 Person Company 2388 Davenport Mountain Apt. 106
Natalieberg, FL 17773 42318 Brian Mews Suite 436
Lucasfurt, NJ 57292 Lindsey Barber Joshua Harris US US 2021-05-17 3 5 6 5000 Person Company 250 Gonzalez Drives
North Jasmine, NE 92676 59707 Christensen Knolls Apt. 250
North Jason, NJ 58845 Bruce White Kathleen Elliott US US 2021-05-18 4 7 8 5000 Person Company 87308 Villanueva Harbor
East Jayberg, MA 20361 81348 Diana Row
Thomasborough, MN 91222 Jeffrey Lara Amy Campbell US US 2021-05-18 5 2 9 4998 Company Company 93324 Hood Islands Apt. 113
Romerofort, UT 96153 2302 Tara Parkways Apt. 683
Higginsland, NJ 53367 Teresa Gibson Mary Strong US US 2021-05-19 6 4 9 4999 Company Company 42318 Brian Mews Suite 436
Lucasfurt, NJ 57292 2302 Tara Parkways Apt. 683
Higginsland, NJ 53367 Joshua Harris Mary Strong US US 2021-05-20 7 6 10 4997 Company Company 59707 Chri

Similar to our de-duplication strategy at an entity level, we can deduplicate multiple fields such as originator and recipient mailing addresses.

In [0]:
settings = {
    "link_type": "dedupe_only",
    "blocking_rules": [
        "l.txn_amount = r.txn_amount",
    ],
    "comparison_columns": [
        
        {
            "col_name": "rptd_originator_address",
        }, 
        { 
            "col_name": "rptd_originator_name",
        }
    ]
}

from splink import Splink
linker = Splink(settings, df2, spark)
df2_e = linker.get_scored_comparisons()

In [0]:
from pyspark.sql.functions import * 
display(df2_e.filter( (col("rptd_originator_address_l") != '')).filter((col("rptd_originator_address_r") != '')))

match_weight match_probability unique_id_l unique_id_r rptd_originator_address_l rptd_originator_address_r gamma_rptd_originator_address rptd_originator_name_l rptd_originator_name_r gamma_rptd_originator_name -2.4528085787437504 0.15444471528086587 223254 223256 Canada Square, Canary Wharf, London, United Kingdom Canada Square, Canary Wharf, London, UK 0 The Bank of New York Mellon Corp. The Bank of New York Mellon Corp. 1 -10.207637653117658 8.449436997835004E-4 1 12046 120 Mason Lake Apt. 211
North Carmenport, OK 73475 Unit 1534 Box 7568
DPO AA 26725 0 Brenda Thomas Adam Hernandez 0 -10.207637653117658 8.449436997835004E-4 1 4 120 Mason Lake Apt. 211
North Carmenport, OK 73475 87308 Villanueva Harbor
East Jayberg, MA 20361 0 Brenda Thomas Jeffrey Lara 0 -10.207637653117658 8.449436997835004E-4 1 3 120 Mason Lake Apt. 211
North Carmenport, OK 73475 250 Gonzalez Drives
North Jasmine, NE 92676 0 Brenda Thomas Bruce White 0 -10.207637653117658 8.449436997835004E-4 1 2 120 Mason Lake Apt. 211
North Carmenport, OK 73475 2388 Davenport Mountain Apt. 106
Natalieberg, FL 17773 0 Brenda Thomas Lindsey Barber 0 -10.207637653117658 8.449436997835004E-4 2 4 2388 Davenport Mountain Apt. 106
Natalieberg, FL 17773 87308 Villanueva Harbor
East Jayberg, MA 20361 0 Lindsey Barber Jeffrey Lara 0 -10.207637653117658 8.449436997835004E-4 2 3 2388 Davenport Mountain Apt. 106
Natalieberg, FL 17773 250 Gonzalez Drives
North Jasmine, NE 92676 0 Lindsey Barber Bruce White 0 -10.207637653117658 8.449436997835004E-4 3 4 250 Gonzalez Drives
North Jasmine, NE 92676 87308 Villanueva Harbor
East Jayberg, MA 20361 0 Bruce White Jeffrey Lara 0 -10.207637653117658 8.449436997835004E-4 10003 12300 0006 Frazier Flat
East Christopherport, ND 80720 29603 Megan Walk Apt. 757
South Aaron, CO 65443 0 Cristian Wilson Donald Baker 0 -10.207637653117658 8.449436997835004E-4 10007 13866 Unit 1267 Box 8698
DPO AP 04962 55787 John Trafficway
New Kelly, AK 14186 0 Lisa Morris Jamie Sims 0 -10.207637653117658 8.449436997835004E-4 10050 13849 872 Tanya Islands Suite 309
Port Michaelchester, SD 81657 39869 Rachel Inlet Apt. 819
Andersonfurt, IA 32378 0 Joseph Johnson Ronald Henry 0 -10.207637653117658 8.449436997835004E-4 10076 17159 83235 Brown Shoal Suite 388
East Todd, MA 15303 8214 Flowers Shoals Apt. 837
Turnerburgh, CA 17591 0 Makayla Reynolds Rodney Wilson 0 -10.207637653117658 8.449436997835004E-4 10080 11481 93115 Roberts Meadows Suite 779
Lake Kimberlyland, IL 36850 7294 Rios Island
Ronaldview, RI 90933 0 Meghan Harris Jessica Whitaker 0 -10.207637653117658 8.449436997835004E-4 10110 11223 45239 Reginald Burg
Leeview, ND 55025 0371 Cobb Neck
Susanside, IL 88273 0 Tony Sanchez Haley Weber 0 -10.207637653117658 8.449436997835004E-4 10114 14024 492 Lin Mountains
Angelicaberg, KS 75025 0339 Lozano Curve Apt. 042
Jessicamouth, ME 76797 0 Sharon Acosta Mikayla Hughes 0 -10.207637653117658 8.449436997835004E-4 10122 13109 607 Richardson Plain
Richardport, MD 57495 4990 Henson Branch Suite 172
South Michelleborough, CA 95552 0 Barbara Vazquez Mary Bolton 0 -10.207637653117658 8.449436997835004E-4 10133 17325 PSC 9061, Box 9398
APO AP 57456 906 Brittany Point Apt. 578
New Alejandrastad, AZ 93143 0 Dr. Shirley West Connie Collins 0 -10.207637653117658 8.449436997835004E-4 10146 12427 696 Wise Causeway Suite 045
Fullerland, AZ 89427 45231 Adkins Camp
South Joshuafort, WV 58923 0 Laura Perez Steven Potter 0 -10.207637653117658 8.449436997835004E-4 10176 11798 56213 William Branch Apt. 817
Rogersfurt, GA 84351 756 Gonzalez Hollow
Port Thomas, TX 03305 0 Katelyn Thompson Nancy Weiss 0 -10.207637653117658 8.449436997835004E-4 10182 10828 2401 Dawn Creek
East Williamshire, VT 72289 85956 Karla Extension Apt. 398
Bennettton, PA 13705 0 Bryan Roth Mr. Jeremy Velez 0 -10.207637653117658 8.449436997835004E-4 10187 11826 51741 Green Hill
Adamville, RI 74521 8508 Carrie Ramp Suite 406
North Kevinborough, PA 88093 0 Barry Williams Patricia Taylor 0 -10.207637653117658 8.449436997835004E-4

## Closing thoughts
In this series of notebooks, we briefly touched on different technical concepts as they pertain to AML investigations. From network analysis, computer vision and entity resolution, we demonstrated the need for AI to complement decisioning and reduce investigation time. Although we kept the investigation aspect light, we covered why the Lakehouse architecture is the most scalable and versatile platform to enable analysts in their AML analytics. All of these capabilities will allow your organization to reduce TCO compared to proprietary AML solutions. Organizations can start embedding advanced analytics in their day to day process by bringing additional context to their investigations team rather than trying to automate what is often regarded as the most regulated activity. AI as **augmented intelligence**, we can easily package all necessary AI driven insights through the form of simple dashboarding capabilities for analysts to act upon.

<img src=https://brysmiwasb.blob.core.windows.net/demos/aml/aml_dashboard.png width=800>